<a href="https://colab.research.google.com/github/goya5858/commonlitreadabilityprize/blob/main/working/BERT_1_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
compe_name = 'commonlitreadabilityprize'
%cd /content/drive/MyDrive/kaggle/works/$compe_name/working/

!pip install -q -U albumentations
!pip install -q -U torch
!pip install -q timm
!pip install -q pytorch_lightning
!pip install -q -U transformers
!pip install -q -U sentencepiece

import os
import re
import gc
import sys
import time
import copy
import warnings
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import PIL.Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

import timm

import nltk
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter
import string

/content/drive/MyDrive/kaggle/works/commonlitreadabilityprize/working
     |████████████████████████████████| 1.2MB 3.6MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Data

In [43]:
#to_replace_by_space = re.compile('[/(){}\[\]|@,;]')
#punctuation = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
#bad_symbols = re.compile('[^0-9a-z #+_]')
#stopwords2 = set(stopwords.words('english'))
#
#def data_preprocessing(text):
#    text = text.lower() 
#    text = re.sub(punctuation, '',text) #[{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’]を削除
#    text = re.sub(to_replace_by_space, " ", text) # [/(){}\[\]|@,;] の文字列群を" "(スペース)で置換
#    text = re.sub(bad_symbols, "", text) #0-9a-z以外と #+_　を削除
#    text = " ".join([word for word in text.split(" ") if word not in stopwords2]) #stopwordsを除外して、spaceで単語をつなげる
#    text = re.sub(' +', ' ', text) # 置き換え
#    return text

#train['excerpt'] = train['excerpt'].apply(data_preprocessing)

# BERT

In [44]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl 
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import model_selection

import transformers
from transformers import get_linear_schedule_with_warmup, AdamW

In [45]:
ROOT = f"../input/{compe_name}/"
df = pd.read_csv(ROOT+'train.csv')
df = df[["id","excerpt","target"]]
df.head()

,id,excerpt,target
0,c12129c31,When the young people returned to the ballroom...,-0.340259
1,85aa80a4c,"All through dinner time, Mrs. Fayre was somewh...",-0.315372
2,b69ac6792,"As Roger had predicted, the snow departed as q...",-0.580118
3,dd1000b26,And outside before the palace a great garden w...,-1.054013
4,37c1b32fb,Once upon a time there were Three Bears who li...,0.247197


In [46]:
test_df = pd.read_csv(ROOT+'test.csv')
test_df.head()

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...


In [47]:
def prep_text(text_df):
  text_df = text_df.str.replace("\n","",regex=False)
  return text_df.str.replace("\'s",r"s",regex=True).values

df['excerpt'] = prep_text(df['excerpt'])
test_df['excerpt'] = prep_text(test_df['excerpt'])

#df['excerpt'] = df['excerpt'].apply(data_preprocessing)
#test_df['excerpt'] = test_df['excerpt'].apply(data_preprocessing)

In [48]:
max_words = df["excerpt"].apply(lambda x: len(x.split())).max()
print("maximum words in instance:",max_words)

maximum words in instance: 205


In [49]:
# trainデータを、targetの値をビニングした値を元に層化fold
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [50]:
df = create_folds(df, num_splits=5)

In [51]:
# BERT Config
BATCH_SIZE = 16
EPOCHS = 15
NUM_TRAIN_STEPS = int((df.shape[0]/BATCH_SIZE)*EPOCHS)
NUM_WARMUP_STEPS = 0
FOLDS = df.kfold.unique()
NUM_FOLDS = df.kfold.nunique() 

MAX_WORDS = df["excerpt"].apply(lambda x: len(x.split())).max()

In [52]:
class BertDataset(Dataset):
  def __init__(self, texts, labels, max_len):
    super().__init__()
    self.texts = texts
    self.labels = labels
    self.max_len = max_len
    self.tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')

  def __len__(self):
    return self.labels.shape[0]
  
  def __getitem__(self, idx):
    text  = " ".join(self.texts.values[idx].split())
    label = self.labels.values[idx]
    inputs = self.tokenizer(text,return_tensors = 'pt', max_length = self.max_len, padding="max_length",truncation=True)
    return {
        "inputs":{"input_ids":inputs["input_ids"][0],
                  "token_type_ids":inputs["token_type_ids"][0],
                  "attention_mask":inputs["attention_mask"][0]},
        "labels": torch.tensor(label, dtype=torch.float)
    }

In [53]:
def get_dataloaders(df, n_fold):
  train_df = df[df['kfold']!=n_fold]
  valid_df = df[df['kfold']==n_fold]

  train_dataset = BertDataset(texts=train_df['excerpt'], labels=train_df['target'], max_len=MAX_WORDS)
  valid_dataset = BertDataset(texts=valid_df['excerpt'], labels=valid_df['target'], max_len=MAX_WORDS)

  train_loader = DataLoader(
      dataset = train_dataset,
      batch_size = BATCH_SIZE,
      shuffle = True,
      #num_workers = -1
    )
  valid_loader = DataLoader(
      dataset = valid_dataset,
      batch_size = BATCH_SIZE,
      shuffle = False,
      #num_workers = -1
    )
  return train_loader, valid_loader

# model

In [54]:
class BERTmodel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = transformers.AutoModel.from_pretrained('bert-base-uncased')
    self.drop = nn.Dropout(0.5)
    self.fc = nn.Linear(in_features=768, out_features=1)
  
  def forward(self,inputs):
    out = self.model(**inputs)
    last_hiddens = out[0]
    out = self.drop(last_hiddens[:,0,:].squeeze(1))
    return self.fc(out)

In [ ]:
#train_loader, valid_loader = get_dataloaders(df, 0)
#
#A = iter(train_loader).next()
#inputs = A['inputs']
#labels = A['labels']
#
#model = BERTmodel()
#model(inputs)

# Training

In [55]:
def train_fn(model, DATA, loss_fn, optim, scheduler):
  optim.zero_grad()
  inputs, labels = DATA['inputs'], DATA['labels'].to(DEVICE)
  inputs = {
      "input_ids":inputs["input_ids"].to(DEVICE),
      "token_type_ids":inputs["token_type_ids"].to(DEVICE),
      "attention_mask":inputs["attention_mask"].to(DEVICE)
  }
  model = model.to(DEVICE)
  pred = model(inputs)
  loss = loss_fn(pred, labels)
  loss.backward()
  optim.step()
  scheduler.step()
  return loss.detach().cpu().numpy()

def valid_fn(model, DATA, loss_fn, optim):
  with torch.no_grad():
    #optim.zero_grad()
    inputs, labels = DATA['inputs'], DATA['labels'].to(DEVICE)
    inputs = {
      "input_ids":inputs["input_ids"].to(DEVICE),
      "token_type_ids":inputs["token_type_ids"].to(DEVICE),
      "attention_mask":inputs["attention_mask"].to(DEVICE)
    }
    model = model.to(DEVICE)
    pred = model(inputs)
    loss = loss_fn(pred, labels)
    #loss.backward()
    #optim.step()
  return loss.detach().cpu().numpy(), pred.cpu().detach().numpy()

In [56]:
# RMSE
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.eps = 1e-8
        
    def forward(self,output,target):
        return torch.sqrt(F.mse_loss(output,target)+self.eps)

metrics = RMSELoss()
def loss_fn(pred, labels, metrics=metrics):
  return metrics(pred.view(-1), labels.view(-1))

In [57]:
def train_fold(n_fold):
  best_score = np.inf
  best_model = []
  best_pred = 0

  train_loader, valid_loader = get_dataloaders(df, n_fold)
  model = BERTmodel().to(DEVICE)
  optimizer = optim.Adam(params=model.parameters(), lr=5e-5)
  lr_scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=len(train_loader))
  metrics = RMSELoss()

  for epoch in range(EPOCHS):
    train_loss = 0
    for DATA in tqdm(train_loader):
      loss = train_fn(model, DATA, loss_fn, optimizer, lr_scheduler)
      train_loss += loss
    train_loss /= len(train_loader)

    valid_loss = 0
    valid_pred = np.ones(1)
    for DATA in tqdm(valid_loader):
      loss, pred = valid_fn(model, DATA, loss_fn, optimizer)
      valid_pred = np.concatenate([valid_pred, pred.reshape(-1,)], axis=0)
      valid_loss += loss
    valid_loss /= len(valid_loader)

    print(f"fold : {n_fold}, epoch : {epoch}, train_loss : {train_loss}")
    print(f"fold : {n_fold}, epoch : {epoch}, valid_loss : {valid_loss}")
    print('='*30)

    if valid_loss < best_score:
      best_socre = valid_loss
      del best_model, best_pred
      gc.collect()
      best_model = copy.deepcopy(model)
      best_pred  = valid_pred[1:]   
  return best_model, best_pred

In [58]:
EPOCHS = 5
DEVICE = 'cuda:0'

oof = np.zeros(df['target'].shape)
best_models = []

for n_fold in range(5):
  best_model, best_pred = train_fold(n_fold)
  best_models.append(best_model)
  oof[(df['kfold']==n_fold)] = best_pred
  print('-='*20)

loss_fn(pred   = torch.tensor(oof).to(DEVICE),
        labels = torch.tensor(df['target'].values).to(DEVICE) ) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



fold : 0, epoch : 0, train_loss : 0.6991841578987282
fold : 0, epoch : 0, valid_loss : 0.5868796722756492



fold : 0, epoch : 1, train_loss : 0.5512863047106166
fold : 0, epoch : 1, valid_loss : 0.6927704910437266



fold : 0, epoch : 2, train_loss : 0.4421178395269622
fold : 0, epoch : 2, valid_loss : 0.5358414840367105


...


fold : 4, epoch : 1, train_loss : 0.5463275258809748
fold : 4, epoch : 1, valid_loss : 0.6503776634732882



fold : 4, epoch : 2, train_loss : 0.4492020855697108
fold : 4, epoch : 2, valid_loss : 0.5490889350573221



fold : 4, epoch : 3, train_loss : 0.39844397801748466
fold : 4, epoch : 3, valid_loss : 0.5677759630812539



fold : 4, epoch : 4, train_loss : 0.3570109091594186
fold : 4, epoch : 4, valid_loss : 0.532365115152465
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


tensor(0.5475, device='cuda:0', dtype=torch.float64)